Create dataset (text file) of bank transactions. Fields in file are ‘Bank ID’, ‘Account Number’,
‘Transaction Date’, ‘Transaction Type’ (credit or debit), ‘Transaction Amount’. Date format is
dd-mm-yyyy.
1. Count unique number of customers
2. Count unique number of Bank ID
3. Count unique number of customers per Bank ID
4. Number of transactions for given Account Number
5. Number of credit transactions for given Account Number in a given year


In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=da372e3e05f189647c6bd9b2aeade38450ebb75f7669bb3d66b708ec8fa1843e
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [8]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('RDD').getOrCreate()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
textRDD=spark.sparkContext.textFile('/content/drive/My Drive/Colab Notebooks/RDD Assignment/bank.txt')
textRDD.collect()

['SBI00042\t123876232\t23-9-2023\tC\t250',
 'SBI00042\t123876342\t13-12-2023\tD\t250',
 'SBI00042\t123876651\t23-1-2024\tC\t250',
 'SBI00042\t123876122\t23-10-2021\tC\t250',
 'SBI00042\t123876122\t23-10-2021\tD\t250',
 'SBI00042\t123876122\t23-10-2021\tC\t250',
 'SBI00042\t123872822\t23-10-2021\tC\t1250',
 'SBI00042\t123876122\t23-10-2021\tD\t250',
 'SBI00042\t123823822\t23-10-2021\tC\t10250',
 'SBI00042\t123876171\t23-10-2021\tD\t25000',
 'SBI00042\t123872322\t23-10-2021\tC\t12500',
 'SBI00042\t123876142\t23-10-2021\tD\t2500',
 'SBI00042\t123872829\t23-10-2021\tC\t100',
 'CAN00123\t720127541\t03-1-2010\tC\t250',
 'CAN00123\t123872822\t23-10-2021\tC\t1250',
 'CAN00123\t123876122\t23-10-2019\tD\t250',
 'CAN00123\t123456789\t23-11-2020\tC\t10250',
 'CAN00123\t123456789\t23-12-2020\tC\t10250',
 'CAN00123\t123456789\t23-10-2020\tC\t10250',
 'CAN00123\t123876171\t13-10-2005\tD\t25000',
 'SBI00042\t123872322\t15-11-2012\tC\t12500',
 'SBI00042\t123876142\t06-01-2006\tD\t2500',
 'SBI00042\t123

In [13]:
dataRdd=textRDD.map(lambda x:x.split('\t'))
dataRdd.collect()

[['SBI00042', '123876232', '23-9-2023', 'C', '250'],
 ['SBI00042', '123876342', '13-12-2023', 'D', '250'],
 ['SBI00042', '123876651', '23-1-2024', 'C', '250'],
 ['SBI00042', '123876122', '23-10-2021', 'C', '250'],
 ['SBI00042', '123876122', '23-10-2021', 'D', '250'],
 ['SBI00042', '123876122', '23-10-2021', 'C', '250'],
 ['SBI00042', '123872822', '23-10-2021', 'C', '1250'],
 ['SBI00042', '123876122', '23-10-2021', 'D', '250'],
 ['SBI00042', '123823822', '23-10-2021', 'C', '10250'],
 ['SBI00042', '123876171', '23-10-2021', 'D', '25000'],
 ['SBI00042', '123872322', '23-10-2021', 'C', '12500'],
 ['SBI00042', '123876142', '23-10-2021', 'D', '2500'],
 ['SBI00042', '123872829', '23-10-2021', 'C', '100'],
 ['CAN00123', '720127541', '03-1-2010', 'C', '250'],
 ['CAN00123', '123872822', '23-10-2021', 'C', '1250'],
 ['CAN00123', '123876122', '23-10-2019', 'D', '250'],
 ['CAN00123', '123456789', '23-11-2020', 'C', '10250'],
 ['CAN00123', '123456789', '23-12-2020', 'C', '10250'],
 ['CAN00123', '123

In [15]:
#1. Count unique number of customers

customer_count=dataRdd.map(lambda x:x[1]).distinct().count()
customer_count

22

In [17]:
#2. Count unique number of Bank ID

unique_count=dataRdd.map(lambda x:x[0]).distinct().count()
unique_count


3

In [23]:
#3.Count unique number of customers per Bank ID

unique_cutomer_per_count=dataRdd.map(lambda x:(x[0],x[1])).distinct().map(lambda d:(d[0],1)).reduceByKey(lambda x,y:x+y)
unique_cutomer_per_count.collect()


[('CAN00123', 8), ('SBI00042', 12), ('ICI00072', 6)]

In [34]:
#4.Number of transactions for given Account Number
#Using BroadCast  method

account_number='123872829'
broadcast_account_number=spark.sparkContext.broadcast(account_number)
print(broadcast_account_number.value)

transactions_for_account = dataRdd.filter(lambda x: x[1] == broadcast_account_number.value)
print(transactions_for_account.collect())

num_transaction=transactions_for_account.count()

print(f"Number of transactions for account {account_number}: {num_transaction}")



123872829
[['SBI00042', '123872829', '23-10-2021', 'C', '100'], ['SBI00042', '123872829', '30-11-2001', 'C', '10000'], ['SBI00042', '123872829', '30-07-2001', 'C', '10000'], ['SBI00042', '123872829', '30-01-2001', 'C', '10000'], ['SBI00042', '123872829', '21-07-2002', 'C', '100'], ['SBI00042', '123872829', '30-01-2001', 'C', '10000'], ['SBI00042', '123872829', '23-05-2024', 'C', '100']]
Number of transactions for account 123872829: 7


In [39]:
#5. Number of credit transactions for given Account Number in a given year

bank_year=dataRdd.map(lambda x:(x[1],x[3],x[2].split('-')[2]))
bank_year.collect()

bank_year.filter(lambda x:(x[0]=='123872829') & (x[1]=='C') & (x[2]=='2001')).count()



4